<a href="https://colab.research.google.com/github/sinbba77/llm-finetuning/blob/main/chapter2_GPT(%EC%A3%BC%EC%84%9D%EC%B6%94%EA%B0%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.1 Data

In [ ]:
# datasets 라이브러리를 설치합니다. 자연어처리 데이터셋을 쉽게 불러올 수 있습니다.
%pip install -q datasets

In [ ]:
# datasets 라이브러리에서 load_dataset 함수를 불러옵니다.
from datasets import load_dataset

# 네이버 뉴스 요약 데이터셋을 불러옵니다.
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [ ]:
# dataset 객체를 data 변수에 저장합니다.
data = dataset
# 데이터셋의 내용을 출력합니다.
data

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [ ]:
# train split의 첫 번째 document(뉴스 기사 원문)를 출력합니다.
data["train"]["document"][0]

'앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.'

In [ ]:
# 첫 번째 뉴스 기사에 등장하는 모든 고유 문자들을 집합으로 만든 후 정렬하여 출력합니다.
print(sorted(list(set(data["train"]["document"][0]))))

[' ', '.', '0', '1', '2', '3', '4', '5', 'E', 'N', 'T', 'U', 'Y', '가', '개', '것', '겠', '격', '견', '겸', '경', '고', '공', '과', '관', '국', '규', '극', '금', '급', '기', '까', '나', '난', '너', '높', '는', '늘', '니', '다', '단', '달', '담', '당', '대', '도', '되', '될', '뒷', '들', '등', '때', '또', '라', '략', '량', '러', '려', '력', '련', '로', '록', '롯', '류', '를', '리', '린', '마', '만', '말', '면', '모', '목', '무', '물', '박', '반', '받', '방', '버', '벌', '보', '복', '본', '부', '비', '산', '상', '서', '선', '성', '세', '소', '속', '쇠', '수', '스', '습', '승', '시', '실', '악', '안', '액', '앵', '야', '양', '억', '업', '에', '엔', '여', '역', '연', '열', '였', '올', '외', '용', '우', '운', '울', '원', '월', '위', '육', '율', '융', '으', '은', '을', '응', '의', '이', '인', '임', '입', '있', '자', '장', '재', '적', '전', '정', '제', '조', '주', '줄', '중', '증', '지', '진', '참', '창', '책', '척', '첨', '체', '초', '총', '최', '추', '출', '침', '커', '케', '크', '통', '투', '특', '팀', '팅', '편', '표', '하', '한', '할', '합', '해', '했', '현', '호', '홍', '화', '확', '환', '황', '회', '획', '효', '히']


In [ ]:
# train split의 모든 뉴스 기사 원문을 하나의 문자열로 합칩니다.
ko_text = "".join(data["train"]["document"])
# 전체 텍스트에서 등장하는 고유 문자들을 집합으로 만든 후 정렬합니다.
ko_chars = sorted(list(set((ko_text))))
# 고유 문자의 개수를 계산합니다.
ko_vocab_size = len(ko_chars)
# 전체 고유 문자 개수를 출력합니다.
print("총 글자 수 :", ko_vocab_size)

총 글자 수 : 2701


In [ ]:
# 고유 문자 리스트에서 2000번째부터 2100번째까지의 문자를 출력합니다.
print(ko_chars[2000:2100])

['왓', '왔', '왕', '왜', '왠', '외', '왹', '왼', '요', '욕', '욘', '욜', '욤', '욥', '용', '우', '욱', '운', '욷', '울', '움', '웁', '웃', '웅', '워', '웍', '원', '월', '웜', '웠', '웡', '웨', '웬', '웰', '웸', '웹', '웻', '위', '윅', '윈', '윌', '윔', '윕', '윗', '윙', '유', '육', '윤', '율', '윱', '윳', '융', '으', '윽', '은', '을', '음', '읍', '읏', '응', '의', '읠', '이', '익', '인', '일', '읽', '잃', '임', '입', '잇', '있', '잉', '잊', '잎', '자', '작', '잔', '잖', '잘', '잠', '잡', '잣', '잤', '장', '잦', '재', '잭', '잰', '잼', '잽', '잿', '쟁', '쟈', '쟝', '쟤', '저', '적', '전', '절']


In [ ]:
# 각 문자에 고유한 id를 부여하는 딕셔너리를 만듭니다.
character_to_ids = {char:i for i, char in enumerate(ko_chars)}
# 각 id에 해당하는 문자를 찾을 수 있는 딕셔너리를 만듭니다.
ids_to_character = {i:char for i, char in enumerate(ko_chars)}
# 문자열을 id 리스트로 변환하는 람다 함수입니다.
token_encode = lambda s:[character_to_ids[c] for c in s]
# id 리스트를 다시 문자열로 변환하는 람다 함수입니다.
token_decode = lambda l: "".join([ids_to_character[i] for i in l])
# 예시 문장을 id 리스트로 변환하여 출력합니다.
print(token_encode("안녕하세요 함께 인공지능을 공부하게 되어 반가워요."))
# 변환된 id 리스트를 다시 문자열로 복원하여 출력합니다.
print(token_decode(token_encode("안녕하세요 함께 인공지능을 공부하게 되어 반가워요.")))

[1909, 1169, 2546, 1770, 2008, 0, 2551, 1061, 0, 2064, 977, 2157, 1209, 2055, 0, 977, 1658, 2546, 949, 0, 1283, 1942, 0, 1593, 908, 2024, 2008, 2]
안녕하세요 함께 인공지능을 공부하게 되어 반가워요.


In [ ]:
# torch 라이브러리를 불러옵니다.
import torch

# 전체 텍스트를 토큰화하여 텐서로 변환합니다.
tokenized_data = torch.tensor(token_encode(ko_text), dtype=torch.long)
# 토큰화된 데이터의 shape과 dtype을 출력합니다.
print(tokenized_data.shape, tokenized_data.dtype)
# 처음 100개의 토큰 id를 출력합니다.
print(tokenized_data[:100])

torch.Size([22162967]) torch.int64
tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987, 2555,    0, 2546, 1593,
        1028,    0, 2015, 1485,    0,  965, 2107, 2060,    0, 1617, 2465, 1542,
        2064,    0, 1808, 2273,    0, 2603, 1236, 1477,    0, 2037, 2555,    0,
        2263, 1430, 2055,    0, 1028, 2019, 2062, 1028, 1441,    0, 2562, 1841,
        1213, 1221,    2,    0, 2451, 2650,    0, 1808, 2273,    0, 2142, 1787,
        1028, 1950, 2060,    0, 1558, 1468, 1119,    0, 2555, 1787, 1477,    0,
        2037, 2555,    0, 1553, 1967, 1024, 2051,    0, 1015, 1541, 1477,    0,
           7,    3, 2117,    0, 2026,    0, 2062, 1740,    0, 2603, 1236, 2546,
         968,    0, 1558, 1468])


In [ ]:
# 학습 데이터와 검증 데이터를 나누기 위한 코드입니다.
# 전체 데이터의 90%를 학습 데이터로 사용합니다.
n = int(0.9 * len(tokenized_data))
# 학습 데이터셋을 만듭니다.
train_dataset = tokenized_data[:n]
# 검증(테스트) 데이터셋을 만듭니다.
test_dataset = tokenized_data[n:]

In [ ]:
# 블록 크기(컨텍스트 길이)를 8로 설정합니다.
block_size = 8
# 학습 데이터셋의 처음 block_size만큼의 데이터를 출력합니다.
train_dataset[:block_size]

tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])

In [ ]:
# 입력 시퀀스와 타겟 시퀀스를 만듭니다.
x = train_dataset[:block_size]
y = train_dataset[1:block_size+1]

# 각 time step마다 context와 target을 출력합니다.
for time in range(block_size):
    # 현재까지의 입력 시퀀스(context)를 만듭니다.
    context = x[:time+1]
    # 현재 time step의 타겟 값을 가져옵니다.
    target = y[time]

    # 입력 텐서와 타겟 글자를 출력합니다.
    print(f"입력 텐셔 : {context}")
    print(f"타켓 글자 : {target}")

입력 텐셔 : tensor([1928])
타켓 글자 : 2315
입력 텐셔 : tensor([1928, 2315])
타켓 글자 : 0
입력 텐셔 : tensor([1928, 2315,    0])
타켓 글자 : 2105
입력 텐셔 : tensor([1928, 2315,    0, 2105])
타켓 글자 : 1658
입력 텐셔 : tensor([1928, 2315,    0, 2105, 1658])
타켓 글자 : 908
입력 텐셔 : tensor([1928, 2315,    0, 2105, 1658,  908])
타켓 글자 : 0
입력 텐셔 : tensor([1928, 2315,    0, 2105, 1658,  908,    0])
타켓 글자 : 1987
입력 텐셔 : tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])
타켓 글자 : 2555


In [ ]:
# 랜덤 시드를 고정하여 실험의 재현성을 높입니다.
torch.manual_seed(1234)

# 배치 크기를 4로 설정합니다.
batch_size = 4
# 블록 크기를 8로 설정합니다.
block_size = 8


# 배치 데이터를 만드는 함수입니다.
def batch_function(mode):
    # 학습 또는 테스트 데이터셋을 선택합니다.
    dataset = train_dataset if mode == "train" else test_dataset
    # 랜덤하게 시작 인덱스를 선택합니다.
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    # 입력 시퀀스를 만듭니다.
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    # 타겟 시퀀스를 만듭니다.
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    # 입력과 타겟을 반환합니다.
    return x, y

# 예시 배치 데이터를 만듭니다.
example_x, example_y = batch_function("train")
# 입력 데이터의 shape을 출력합니다.
print("inputs : ", example_x.shape)
print("")
# 실제 입력 데이터 값을 출력합니다.
print("example_x의 실제 값")
print(example_x)
print("-----------------------")
# 타겟 데이터의 shape을 출력합니다.
print("targets : ", example_y.shape)
print("")
# 실제 타겟 데이터 값을 출력합니다.
print("example_y의 실제 값")
print(example_y)
print("-----------------------")

# 각 배치와 시퀀스 위치마다 context와 target을 출력합니다.
for size in range(batch_size):
    for t in range(block_size):
        # 현재까지의 입력 시퀀스(context)를 만듭니다.
        context = example_x[size, :t+1]
        # 현재 위치의 타겟 값을 가져옵니다.
        target = example_y[size, t]
        print(f"input : {context}, target : {target}")
    print("-----------------------")
    print("-----------------------")

inputs :  torch.Size([4, 8])

example_x의 실제 값
tensor([[1764, 2555,    0, 1236, 2248,    0, 2017, 1976],
        [   0, 1966, 2157,    0, 1951, 2062,    0, 2548],
        [   0, 1304, 1485, 1586,    0, 1907, 2450,    0],
        [   3,    2,    6,    5,    1,    0,    5,    3]])
-----------------------
targets :  torch.Size([4, 8])

example_y의 실제 값
tensor([[2555,    0, 1236, 2248,    0, 2017, 1976, 2546],
        [1966, 2157,    0, 1951, 2062,    0, 2548, 2289],
        [1304, 1485, 1586,    0, 1907, 2450,    0, 2480],
        [   2,    6,    5,    1,    0,    5,    3,    5]])
-----------------------
input : tensor([1764]), target : 2555
input : tensor([1764, 2555]), target : 0
input : tensor([1764, 2555,    0]), target : 1236
input : tensor([1764, 2555,    0, 1236]), target : 2248
input : tensor([1764, 2555,    0, 1236, 2248]), target : 0
input : tensor([1764, 2555,    0, 1236, 2248,    0]), target : 2017
input : tensor([1764, 2555,    0, 1236, 2248,    0, 2017]), target : 1976
input :

In [ ]:
# 전체 vocab 크기, 입력 데이터 shape, 타겟 데이터 shape을 출력합니다.
ko_vocab_size, example_x.shape, example_y

(2701,
 torch.Size([4, 8]),
 tensor([[2555,    0, 1236, 2248,    0, 2017, 1976, 2546],
         [1966, 2157,    0, 1951, 2062,    0, 2548, 2289],
         [1304, 1485, 1586,    0, 1907, 2450,    0, 2480],
         [   2,    6,    5,    1,    0,    5,    3,    5]]))

# 2.3 언어모델 만들기

## 2.3.1 ~ 2.3.2 라이브러리 설명 & _ _ init _ _ 함수

## 참고
1. 임베딩 차원 설정: 현재 코드에서는 임베딩 차원을 vocab_length와 동일하게 설정했는데, 실제로는 보통 더 작은 값(예: 128, 256, 512)을 사용합니다.
2. targets 매개변수: forward 함수에서 targets를 받고 있지만 실제로는 사용하지 않고 있습니다. 일반적으로는 손실 계산 시에 사용됩니다.
3. 모델 구조: 이는 매우 간단한 GPT 모델로, 실제 GPT에는 어텐션 메커니즘, 다중 레이어, 위치 인코딩 등이 추가로 필요합니다.

In [ ]:
# PyTorch의 핵심 라이브러리를 import합니다. torch는 텐서 연산과 GPU 가속을 위한 기본 라이브러리입니다.
import torch

# 신경망 구축을 위한 PyTorch의 neural network 모듈을 import합니다.
import torch.nn as nn

# 활성화 함수, 손실 함수 등 다양한 기능 함수들이 포함된 functional 모듈을 import합니다.
from torch.nn import functional as F

# semiGPT라는 이름의 커스텀 신경망 클래스를 정의합니다. nn.Module을 상속받아 PyTorch 모델의 기본 구조를 따릅니다.
class semiGPT(nn.Module):

    # 클래스의 생성자(__init__)를 정의합니다. vocab_length는 어휘 사전의 크기를 나타내는 매개변수입니다.
    def __init__(self, vocab_length):

        # 부모 클래스(nn.Module)의 생성자를 호출하여 기본 초기화를 수행합니다.
        super().__init__()

        # 임베딩 테이블을 생성합니다. 첫 번째 매개변수는 어휘 사전 크기, 두 번째는 임베딩 차원입니다.
        # 여기서는 둘 다 vocab_length로 설정했는데, 일반적으로는 임베딩 차원을 더 작게 설정합니다.
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    # 순전파(forward pass) 함수를 정의합니다. inputs는 입력 토큰 인덱스들, targets는 정답 레이블입니다.
    def forward(self, inputs, targets):

        # 입력 토큰들을 임베딩 테이블을 통해 벡터로 변환합니다.
        # inputs의 각 토큰 인덱스가 해당하는 임베딩 벡터로 매핑됩니다.
        logits = self.embedding_token_table(inputs)

        # 변환된 임베딩 벡터들(logits)을 반환합니다. 이는 각 토큰의 예측 확률 분포를 나타냅니다.
        return logits

# semiGPT 모델의 인스턴스를 생성합니다. ko_vocab_size는 한국어 어휘 사전의 크기를 나타내는 변수입니다.
model = semiGPT(ko_vocab_size)

# 생성된 모델에 예시 데이터(example_x, example_y)를 입력하여 순전파를 수행합니다.
output = model(example_x, example_y)

# 모델의 출력 텐서의 형태(shape)를 출력합니다. 일반적으로 (배치 크기, 시퀀스 길이, 어휘 사전 크기) 형태입니다.
print(output.shape)

torch.Size([4, 8, 2701])


In [ ]:
#에러가 발생되도록 설정한 코드
# nn.Embedding(num_embeddings=4, embedding_dim=4)은 총 **4개의 임베딩 벡터(인덱스 0~3)**만 정의됩니다.
embedding = nn.Embedding(4, 4)

# embedding(torch.tensor([[0, 1, 2, 10]]))는 인덱스 10을 요청하고 있는데, 이는 허용 범위를 벗어납니다.
embedding(torch.tensor([[0, 1, 2, 10]]))

IndexError: index out of range in self

In [17]:
#에러가 발생되도록 세팅된 코드
import torch
import torch.nn as nn
from torch.nn import functional as F

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets):
        logits = self.embedding_token_table(inputs)

        loss = F.cross_entropy(logits, targets)
        return logits, loss

model = semiGPT(ko_vocab_size)
output, loss = model(example_x, example_y)
print(output)

RuntimeError: Expected target size [4, 2701], got [4, 8]

## 2.3.3 forward 메서드

## 변경사항
1. 텐서 재구성 (Reshaping): view() 함수를 사용해 3차원 텐서를 2차원으로, 2차원 텐서를 1차원으로 변환하는 이유는 PyTorch의 cross_entropy 함수가 특정 형태의 입력을 요구하기 때문입니다.
2. 교차 엔트로피 손실: 언어 모델에서 가장 일반적으로 사용되는 손실 함수로, 예측 확률 분포와 실제 정답 간의 차이를 측정합니다.
3. 배치 처리: 여러 시퀀스를 동시에 처리하기 위해 배치 차원을 고려한 텐서 조작이 이루어집니다.
4. 디버깅 출력: 텐서의 형태를 확인하는 것은 딥러닝 개발에서 매우 중요한 디버깅 기법입니다.

In [19]:
# PyTorch의 핵심 라이브러리를 import합니다. torch는 텐서 연산과 GPU 가속을 위한 기본 라이브러리입니다.
import torch

# 신경망 구축을 위한 PyTorch의 neural network 모듈을 import합니다.
import torch.nn as nn

# 활성화 함수, 손실 함수 등 다양한 기능 함수들이 포함된 functional 모듈을 import합니다.
from torch.nn import functional as F

# semiGPT라는 이름의 커스텀 신경망 클래스를 정의합니다. nn.Module을 상속받아 PyTorch 모델의 기본 구조를 따릅니다.
class semiGPT(nn.Module):

    # 클래스의 생성자(__init__)를 정의합니다. vocab_length는 어휘 사전의 크기를 나타내는 매개변수입니다.
    def __init__(self, vocab_length):

        # 부모 클래스(nn.Module)의 생성자를 호출하여 기본 초기화를 수행합니다.
        super().__init__()

        # 임베딩 테이블을 생성합니다. 첫 번째 매개변수는 어휘 사전 크기, 두 번째는 임베딩 차원입니다.
        # 여기서는 둘 다 vocab_length로 설정했는데, 일반적으로는 임베딩 차원을 더 작게 설정합니다.
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    # 순전파(forward pass) 함수를 정의합니다. inputs는 입력 토큰 인덱스들, targets는 정답 레이블입니다.
    def forward(self, inputs, targets):

        # 입력 토큰들을 임베딩 테이블을 통해 벡터로 변환합니다.
        # inputs의 각 토큰 인덱스가 해당하는 임베딩 벡터로 매핑됩니다.
        logits = self.embedding_token_table(inputs)

        # logits 텐서의 차원을 추출합니다. batch는 배치 크기, seq_length는 시퀀스 길이, vocab_length는 어휘 사전 크기입니다.
        batch, seq_length, vocab_length = logits.shape

        # 손실 함수 계산을 위해 logits를 2차원으로 재구성합니다. (batch_size * seq_length, vocab_size) 형태로 변환합니다.
        # 이는 cross_entropy 함수가 2차원 입력을 요구하기 때문입니다.
        logits = logits.view(batch * seq_length, vocab_length)

        # targets도 1차원으로 재구성합니다. (batch_size * seq_length,) 형태로 변환합니다.
        # 각 위치의 정답 토큰 인덱스가 일렬로 나열됩니다.
        targets = targets.view(batch*seq_length)

        # 교차 엔트로피 손실(cross entropy loss)을 계산합니다. 이는 분류 문제에서 가장 일반적으로 사용되는 손실 함수입니다.
        # logits는 예측 확률 분포, targets는 정답 레이블입니다.
        loss = F.cross_entropy(logits, targets)

        # 디버깅을 위해 logits의 형태를 출력합니다. 재구성 후의 2차원 텐서 형태를 확인할 수 있습니다.
        print("logits의 shape는 : ", logits.shape, "입니다.")

        # 디버깅을 위해 targets의 형태를 출력합니다. 재구성 후의 1차원 텐서 형태를 확인할 수 있습니다.
        print("targets의 shape는 : ", targets.shape, "입니다.")

        # 예측 결과(logits)와 계산된 손실값(loss)을 튜플 형태로 반환합니다.
        return logits, loss

# semiGPT 모델의 인스턴스를 생성합니다. ko_vocab_size는 한국어 어휘 사전의 크기를 나타내는 변수입니다.
model = semiGPT(ko_vocab_size)

# 생성된 모델에 예시 데이터(example_x, example_y)를 입력하여 순전파를 수행하고, 결과를 logits와 loss로 받습니다.
logits, loss = model(example_x, example_y)

# 계산된 손실값을 출력합니다. 이 값이 작을수록 모델의 예측이 정답에 가깝다는 의미입니다.
print(loss)

logits의 shape는 :  torch.Size([32, 2701]) 입니다.
targets의 shape는 :  torch.Size([32]) 입니다.
tensor(8.4701, grad_fn=<NllLossBackward0>)


In [20]:
example_x.shape, example_y.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

## 2.3.4 generate 메서드

## 주요 개념과 기능들:

1. 조건부 손실 계산: targets=None을 통해 학습 모드와 추론 모드를 구분합니다.
2. 자동회귀 생성: generate() 함수는 한 번에 하나씩 토큰을 생성하여 시퀀스를 확장해나가는 GPT의 핵심 메커니즘입니다.
3. 확률적 샘플링: torch.multinomial()을 사용하여 확률 분포에서 다음 토큰을 랜덤하게 선택합니다. 이는 생성된 텍스트에 다양성을 부여합니다.
4. 시퀀스 확장: torch.cat()을 사용하여 기존 시퀀스에 새 토큰을 추가합니다.
5. 마지막 위치 선택: logits[:, -1, :]는 시퀀스의 마지막 위치만을 사용하여 다음 토큰을 예측하는 GPT의 특징입니다.

In [21]:
# PyTorch의 핵심 라이브러리를 import합니다. torch는 텐서 연산과 GPU 가속을 위한 기본 라이브러리입니다.
import torch

# 신경망 구축을 위한 PyTorch의 neural network 모듈을 import합니다.
import torch.nn as nn

# 활성화 함수, 손실 함수 등 다양한 기능 함수들이 포함된 functional 모듈을 import합니다.
from torch.nn import functional as F

# semiGPT라는 이름의 커스텀 신경망 클래스를 정의합니다. nn.Module을 상속받아 PyTorch 모델의 기본 구조를 따릅니다.
class semiGPT(nn.Module):

    # 클래스의 생성자(__init__)를 정의합니다. vocab_length는 어휘 사전의 크기를 나타내는 매개변수입니다.
    def __init__(self, vocab_length):

        # 부모 클래스(nn.Module)의 생성자를 호출하여 기본 초기화를 수행합니다.
        super().__init__()

        # 임베딩 테이블을 생성합니다. 첫 번째 매개변수는 어휘 사전 크기, 두 번째는 임베딩 차원입니다.
        # 여기서는 둘 다 vocab_length로 설정했는데, 일반적으로는 임베딩 차원을 더 작게 설정합니다.
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    # 순전파(forward pass) 함수를 정의합니다. inputs는 입력 토큰 인덱스들, targets는 정답 레이블이며 기본값은 None입니다.
    def forward(self, inputs, targets=None):

        # 입력 토큰들을 임베딩 테이블을 통해 벡터로 변환합니다.
        # inputs의 각 토큰 인덱스가 해당하는 임베딩 벡터로 매핑됩니다.
        logits = self.embedding_token_table(inputs)

        # targets가 None인 경우 (추론/생성 모드)에는 손실을 계산하지 않습니다.
        if targets is None:
            loss = None
        else:
            # 학습 모드에서는 손실을 계산합니다.
            # logits 텐서의 차원을 추출합니다. batch는 배치 크기, seq_length는 시퀀스 길이, vocab_length는 어휘 사전 크기입니다.
            batch, seq_length, vocab_length = logits.shape

            # 손실 함수 계산을 위해 logits를 2차원으로 재구성합니다. (batch_size * seq_length, vocab_size) 형태로 변환합니다.
            logits = logits.view(batch * seq_length, vocab_length)

            # targets도 1차원으로 재구성합니다. (batch_size * seq_length,) 형태로 변환합니다.
            targets = targets.view(batch*seq_length)

            # 교차 엔트로피 손실(cross entropy loss)을 계산합니다. 이는 분류 문제에서 가장 일반적으로 사용되는 손실 함수입니다.
            loss = F.cross_entropy(logits, targets)

        # 예측 결과(logits)와 계산된 손실값(loss)을 튜플 형태로 반환합니다.
        return logits, loss

    # 텍스트 생성을 위한 함수입니다. inputs는 시작 토큰들, max_new_tokens는 생성할 새로운 토큰의 최대 개수입니다.
    def generate(self, inputs, max_new_tokens):

        # 지정된 개수만큼 새로운 토큰을 반복적으로 생성합니다.
        for _ in range(max_new_tokens):

            # 현재 입력에 대해 순전파를 수행하여 다음 토큰의 확률 분포를 얻습니다.
            logits, loss = self.forward(inputs)

            # 마지막 위치의 토큰에 대한 예측만을 사용합니다. [:, -1, :]는 모든 배치의 마지막 시퀀스 위치를 선택합니다.
            logits = logits[:, -1, :]

            # 디버깅을 위해 logits의 형태를 출력합니다. (batch_size, vocab_size) 형태여야 합니다.
            print(logits.shape)

            # logits를 확률 분포로 변환합니다. softmax 함수를 사용하여 각 토큰의 선택 확률을 계산합니다.
            probs = F.softmax(logits, dim=-1)

            # 확률 분포에서 다음 토큰을 샘플링합니다. multinomial은 확률에 따라 랜덤하게 토큰을 선택합니다.
            # num_samples=1은 하나의 토큰만 선택한다는 의미입니다.
            next_inputs = torch.multinomial(probs, num_samples=1)

            # 기존 입력 시퀀스에 새로 생성된 토큰을 연결합니다. dim=1은 시퀀스 차원을 따라 연결한다는 의미입니다.
            inputs = torch.cat((inputs, next_inputs), dim=1)

        # 생성이 완료된 전체 시퀀스를 반환합니다.
        return inputs

# semiGPT 모델의 인스턴스를 생성합니다. ko_vocab_size는 한국어 어휘 사전의 크기를 나타내는 변수입니다.
model = semiGPT(ko_vocab_size)

# 생성된 모델에 예시 데이터(example_x, example_y)를 입력하여 순전파를 수행하고, 결과를 logits와 loss로 받습니다.
logits, loss = model(example_x, example_y)

# 계산된 손실값을 출력합니다. 이 값이 작을수록 모델의 예측이 정답에 가깝다는 의미입니다.
print(loss)

# 텍스트 생성을 수행합니다.
# torch.zeros((1,1), dtype=torch.long)는 크기가 1x1인 0으로 채워진 정수 텐서를 생성합니다 (시작 토큰).
# max_new_tokens=10은 10개의 새로운 토큰을 생성한다는 의미입니다.
# [0]는 배치 차원을 제거하고, .tolist()는 텐서를 파이썬 리스트로 변환합니다.
# token_decode는 토큰 인덱스들을 실제 텍스트로 변환하는 함수입니다 (별도로 정의되어 있어야 함).
token_decode(model.generate(torch.zeros((1,1),
                                       dtype=torch.long),
                           max_new_tokens=10)[0].tolist())

tensor(8.1922, grad_fn=<NllLossBackward0>)
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])


' 훼팅곧©자뺐멘햅켰깔'

In [22]:
# PyTorch 라이브러리를 import합니다. 텐서 연산과 딥러닝을 위한 핵심 라이브러리입니다.
import torch

# 3차원 텐서를 생성합니다. 형태는 (1, 3, 4)입니다.
# - 첫 번째 차원(1): 배치 크기 (batch size)
# - 두 번째 차원(3): 시퀀스 길이 (sequence length) - 3개의 토큰/단어
# - 세 번째 차원(4): 어휘 사전 크기 (vocabulary size) - 4개의 가능한 토큰
logits = torch.tensor(
    [
        [
            [0.1, 0.2, 0.3, 0.4],  # 첫 번째 토큰의 예측 확률 분포
            [0.2, 0.3, 0.4, 0.1],  # 두 번째 토큰의 예측 확률 분포
            [0.3, 0.4, 0.1, 0.2]   # 세 번째 토큰의 예측 확률 분포 (마지막 토큰)
        ]
    ]
)

# 텐서 슬라이싱을 수행합니다.
# [:, -1, :] 의미:
# - 첫 번째 (:): 모든 배치를 선택 (여기서는 배치가 1개뿐)
# - 두 번째 (-1): 시퀀스의 마지막 위치를 선택 (-1은 마지막 인덱스를 의미)
# - 세 번째 (:): 해당 위치의 모든 어휘 확률값을 선택
# 결과적으로 마지막 토큰 위치의 예측 확률 분포만을 추출합니다.
result = logits[:,-1,:]

# 선택된 결과값을 출력합니다. 마지막 토큰의 확률 분포 [0.3, 0.4, 0.1, 0.2]가 출력됩니다.
print("선택되는 값        : ", result)

# 결과 텐서의 크기(shape)를 출력합니다. torch.Size([1, 4])가 출력됩니다.
# - 첫 번째 차원(1): 배치 크기가 유지됨
# - 두 번째 차원(4): 어휘 사전 크기가 유지됨
# - 시퀀스 차원은 마지막 위치만 선택했으므로 제거됨
print("결과에 대한 size 값 : ", result.size())

선택되는 값        :  tensor([[0.3000, 0.4000, 0.1000, 0.2000]])
결과에 대한 size 값 :  torch.Size([1, 4])


## 2.4 optimizer 추가하기

In [23]:
# 학습률(learning rate)을 설정합니다. 1e-2는 0.01을 의미하는 과학적 표기법입니다.
# 학습률은 경사하강법에서 매개변수를 업데이트할 때 얼마나 큰 폭으로 이동할지를 결정하는 하이퍼파라미터입니다.
# 너무 크면 최적값을 지나칠 수 있고, 너무 작으면 학습이 매우 느려집니다.
learning_rate = 1e-2

# semiGPT 모델의 인스턴스를 생성합니다.
# ko_vocab_size는 한국어 어휘 사전의 크기를 나타내는 변수로, 모델이 처리할 수 있는 토큰의 종류 수입니다.
model = semiGPT(ko_vocab_size)

# AdamW 옵티마이저를 생성합니다.
# - AdamW는 Adam 옵티마이저의 개선된 버전으로, weight decay를 더 효과적으로 처리합니다.
# - Adam은 적응적 학습률을 사용하는 최적화 알고리즘으로, 각 매개변수마다 다른 학습률을 적용합니다.
# - model.parameters()는 모델의 모든 학습 가능한 매개변수(가중치와 편향)를 반환합니다.
# - lr=learning_rate로 앞서 정의한 학습률을 설정합니다.
# AdamW는 현재 트랜스포머 모델(GPT, BERT 등)에서 가장 널리 사용되는 옵티마이저입니다.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## 학습 루프의 4단계:

1. 데이터 로딩: batch_function()으로 학습 데이터 배치를 가져옴
2. 순전파: 모델에 데이터를 입력하여 예측값과 손실 계산
3. 역전파: loss.backward()로 기울기 계산
4. 매개변수 업데이트: optimizer.step()으로 가중치 갱신

In [24]:
# tqdm 라이브러리에서 auto 모듈을 import합니다.
# tqdm은 반복문의 진행 상황을 시각적으로 보여주는 진행 표시줄(progress bar)을 제공합니다.
# auto는 환경에 따라 자동으로 적절한 tqdm 버전을 선택합니다 (Jupyter 노트북 vs 터미널).
from tqdm.auto import tqdm

# 배치 크기를 32로 설정합니다.
# 배치 크기는 한 번에 처리할 샘플의 개수를 의미하며, GPU 메모리와 학습 안정성을 고려하여 설정합니다.
# 32는 일반적으로 사용되는 중간 크기의 배치 사이즈입니다.
batch_size = 32

# 10,000번의 학습 스텝을 수행하는 메인 학습 루프입니다.
# tqdm()으로 감싸서 진행 상황을 시각적으로 확인할 수 있습니다.
for steps in tqdm(range(10000)):

    # batch_function을 호출하여 학습용 데이터 배치를 가져옵니다.
    # "train"은 학습 데이터셋을 사용한다는 의미입니다.
    # example_x는 입력 데이터(토큰 시퀀스), example_y는 정답 레이블(다음 토큰)입니다.
    example_x, example_y = batch_function("train")

    # 모델에 입력 데이터를 전달하여 순전파(forward pass)를 수행합니다.
    # logits는 각 토큰에 대한 예측 확률 분포, loss는 예측과 정답 간의 오차입니다.
    logits, loss = model(example_x, example_y)

    # 옵티마이저의 기울기(gradient)를 초기화합니다.
    # PyTorch는 기본적으로 기울기를 누적하므로, 매 스텝마다 이전 기울기를 제거해야 합니다.
    # set_to_none=True는 메모리 효율성을 위해 기울기를 None으로 설정합니다 (0으로 설정하는 것보다 빠름).
    optimizer.zero_grad(set_to_none=True)

    # 역전파(backward pass)를 수행하여 각 매개변수에 대한 기울기를 계산합니다.
    # 이는 연쇄 법칙(chain rule)을 사용하여 손실 함수로부터 모든 매개변수의 기울기를 자동으로 계산합니다.
    loss.backward()

    # 계산된 기울기를 사용하여 모델의 매개변수(가중치)를 업데이트합니다.
    # AdamW 옵티마이저가 적응적 학습률과 모멘텀을 적용하여 매개변수를 최적화합니다.
    optimizer.step()

# 학습이 완료된 후 마지막 손실값을 출력합니다.
# .item()은 텐서에서 파이썬 숫자 값을 추출하는 메서드입니다.
# 이 값이 작을수록 모델이 데이터를 잘 학습했다는 의미입니다.
print(loss.item())

  0%|          | 0/10000 [00:00<?, ?it/s]

3.428206443786621


In [39]:
# print(token_decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=10)[0].tolist()))

# 학습된 모델을 사용하여 텍스트를 생성하고 결과를 출력합니다.
# 이는 모델이 얼마나 잘 학습되었는지 확인하는 테스트 과정입니다.

print(
    # token_decode 함수를 사용하여 생성된 토큰 인덱스들을 실제 텍스트로 변환합니다.
    # 이 함수는 숫자로 된 토큰 ID들을 사람이 읽을 수 있는 문자열로 디코딩합니다.
    token_decode(
        # model.generate() 함수를 호출하여 텍스트를 생성합니다.
        model.generate(
            # 시작 토큰으로 0을 사용합니다.
            # torch.zeros((1,1), dtype=torch.long)는 크기가 1x1인 정수 텐서를 생성합니다.
            # - (1,1): 배치 크기 1, 시퀀스 길이 1을 의미
            # - dtype=torch.long: 정수 타입으로 설정 (토큰 인덱스는 정수여야 함)
            # - 값 0: 보통 특수 토큰(패딩, 시작 토큰 등)으로 사용되는 인덱스
            torch.zeros((1,1), dtype=torch.long),

            # 생성할 새로운 토큰의 개수를 10개로 설정합니다.
            # 시작 토큰 1개 + 새로 생성할 토큰 10개 = 총 11개의 토큰 시퀀스가 생성됩니다.
            max_new_tokens=10
        )
        # generate() 함수는 배치 차원을 포함한 텐서를 반환하므로 [0]으로 첫 번째 배치를 선택합니다.
        [0]
        # PyTorch 텐서를 파이썬 리스트로 변환합니다.
        # token_decode 함수가 리스트 형태의 토큰 인덱스를 입력으로 받기 때문입니다.
        .tolist()
    )
)

torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
torch.Size([1, 2701])
 험 한다. 준비금리


### 2.4.1 GPU로 데이터를 옮기는 방법

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가
    return x, y

### 2.4.2 Loss 함수 만들기

In [ ]:
@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

### 2.4.3 전체 코드 복습

* 책에는 for문부터 설명이 나와있지만, 실행하는데 햇갈리실거 같아 코드는 전체 코드를 기록해놓았습니다.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200

def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가

@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets=None):
        logits = self.embedding_token_table(inputs)
        if targets is None:
            loss = None
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs

model = semiGPT(ko_vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 8.3225, val loss : 8.3266
step : 300, train loss : 6.0857, val loss : 6.0748
step : 600, train loss : 4.7943, val loss : 4.7822
step : 900, train loss : 4.2353, val loss : 4.2261
step : 1200, train loss : 3.9576, val loss : 3.9557
step : 1500, train loss : 3.8215, val loss : 3.8180
step : 1800, train loss : 3.7062, val loss : 3.7000
step : 2100, train loss : 3.6504, val loss : 3.6657
step : 2400, train loss : 3.6248, val loss : 3.6283
step : 2700, train loss : 3.5896, val loss : 3.5865
step : 3000, train loss : 3.5797, val loss : 3.5695
step : 3300, train loss : 3.5399, val loss : 3.5308
step : 3600, train loss : 3.5256, val loss : 3.5289
step : 3900, train loss : 3.5061, val loss : 3.5127
step : 4200, train loss : 3.4900, val loss : 3.4989
step : 4500, train loss : 3.4810, val loss : 3.4903
step : 4800, train loss : 3.4961, val loss : 3.4815
step : 5100, train loss : 3.4668, val loss : 3.4674
step : 5400, train loss : 3.4622, val loss : 3.4567
step : 5700, train

## 2.5 설프 어텐션 추가하기

### 2.5.1	문자들 간의 정보를 주고받는 방식(평균 방식)

In [ ]:
import torch
torch.manual_seed(1441)
num_batches, sequence_length, embedding_dim = 2, 4, 6
embeddings_tensor = torch.randn(num_batches,
                                sequence_length,
                                embedding_dim)
embeddings_tensor.shape

torch.Size([2, 4, 6])

In [ ]:
# 이전 임베딩의 평균을 저장할 텐서 초기화
averaged_embeddings = torch.zeros((num_batches, sequence_length, embedding_dim))

# 각 배치에 대해 반복
for batch_index in range(num_batches):
    # 각 시퀀스 위치에 대해 반복
    for sequence_position in range(sequence_length):
        # 현재 시퀀스 위치까지의 이전 임베딩을 슬라이스
        previous_embeddings = embeddings_tensor[batch_index, :sequence_position + 1]
        # 현재 위치까지의 임베딩의 평균을 계산
        averaged_embeddings[batch_index, sequence_position] = torch.mean(
            previous_embeddings,
            dim=0
        )

In [ ]:
print(embeddings_tensor[0])
print(averaged_embeddings[0])

tensor([[-1.1437, -1.2611, -0.1634, -0.5255, -1.0879,  0.3712],
        [ 2.2335,  0.3099, -1.3975,  1.1141, -0.3373,  0.6924],
        [ 0.2644,  1.1567, -0.5040, -0.7986,  2.6778,  1.4161],
        [ 1.3159, -0.5231,  1.2933, -0.8819,  0.7118,  0.4209]])
tensor([[-1.1437, -1.2611, -0.1634, -0.5255, -1.0879,  0.3712],
        [ 0.5449, -0.4756, -0.7804,  0.2943, -0.7126,  0.5318],
        [ 0.4514,  0.0685, -0.6883, -0.0700,  0.4175,  0.8266],
        [ 0.6675, -0.0794, -0.1929, -0.2730,  0.4911,  0.7252]])


In [ ]:
print(embeddings_tensor[0][0])
print(averaged_embeddings[0][0])

tensor([-1.1437, -1.2611, -0.1634, -0.5255, -1.0879,  0.3712])
tensor([-1.1437, -1.2611, -0.1634, -0.5255, -1.0879,  0.3712])


In [ ]:
print(embeddings_tensor[0][1])
print(averaged_embeddings[0][1])

tensor([ 2.2335,  0.3099, -1.3975,  1.1141, -0.3373,  0.6924])
tensor([ 0.5449, -0.4756, -0.7804,  0.2943, -0.7126,  0.5318])


In [ ]:
(embeddings_tensor[0][0][0] + averaged_embeddings[0][1][0]) / 2

tensor(0.5449)

### 2.5.2. 행렬곱 연산으로 더 빠르게 정보를 주고받기

In [ ]:
# 행렬곱 연산 예시

A = torch.ones(3,3)
B = torch.randint(0, 10, (3,2)).float()
AB = A @ B

print(" A 행렬 ")
print(A)
print("==============")
print("==============")
print(" B 행렬 ")
print(B)
print("==============")
print("==============")
print(" AB 행렬 ")
print(AB)

 A 행렬 
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
 B 행렬 
tensor([[7., 2.],
        [0., 5.],
        [2., 2.]])
 AB 행렬 
tensor([[9., 9.],
        [9., 9.],
        [9., 9.]])


tril이라는 torch의 함수를 이용해서 구해보겠습니다.

In [ ]:
weight = torch.tril(torch.ones(sequence_length, sequence_length))
print(weight)
weight = weight / weight.sum(1, keepdim=True)
print(weight)

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])


In [ ]:
matrix_averaged_embeddings = weight @ embeddings_tensor
torch.allclose(averaged_embeddings, matrix_averaged_embeddings)

True

In [ ]:
weight = torch.tril(torch.ones(sequence_length, sequence_length))
weight = weight.masked_fill(weight == 0, float('-inf')) # 0이라는 숫자에는 -inf를 쓰우겠다는 코드이다.
print(weight)
weight = F.softmax(weight, dim=-1)
print(weight)

tensor([[1., -inf, -inf, -inf],
        [1., 1., -inf, -inf],
        [1., 1., 1., -inf],
        [1., 1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])


In [ ]:
weight_tril_embeddings = weight @ embeddings_tensor
torch.allclose(averaged_embeddings, weight_tril_embeddings)

True

이렇게 맨마지막 layer까지 평균 정보를 전달할 수 있습니다.  
또한, 지금은 weight가 모두 0이라서 softmax를 한 후 동일한 값을 같는 것을 볼 수 있는데 실제 데이터에서는 서로 연관성이 높은 것들을 찾을 것입니다.

### 2.5.3 셀프 어텐션이란?


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 고정된 난수 시드 설정
torch.manual_seed(1111)

# 배치 크기, 시퀀스 길이, 채널 수 설정
batch_size, seq_length, num_channels = 2, 4, 4
input_tensor = torch.randn(batch_size, seq_length, num_channels)

# 각 헤드의 크기
head_size = 16

# Key, Query, Value 변환을 위한 선형 레이어
key_transform = nn.Linear(num_channels, head_size, bias=False)
query_transform = nn.Linear(num_channels, head_size, bias=False)
value_transform = nn.Linear(num_channels, head_size, bias=False)

# Key, Query, Value 변환 수행
keys = key_transform(input_tensor)
queries = query_transform(input_tensor)
values = value_transform(input_tensor)

# Attention 스코어 계산
attention_scores = queries @ keys.transpose(-2, -1)

# 하삼각행렬 생성 및 마스킹
mask_lower_triangle = torch.tril(torch.ones(seq_length, seq_length))
attention_scores = attention_scores.masked_fill(mask_lower_triangle == 0, float('-inf'))

# 소프트맥스 함수를 사용하여 확률 정규화
normalized_scores = F.softmax(attention_scores, dim=-1)

# 최종 출력 계산
output_tensor = normalized_scores @ values

output_tensor


tensor([[[-0.4755, -0.5409, -0.1864,  0.2951, -1.0717, -0.6172, -0.0176,
           0.1793, -0.1113,  0.6589, -0.4507, -0.1181, -0.9728, -0.8870,
           0.2349, -0.0431],
         [-0.4675, -0.5344, -0.1847,  0.2859, -1.0581, -0.6044, -0.0154,
           0.1778, -0.1141,  0.6524, -0.4473, -0.1211, -0.9561, -0.8733,
           0.2352, -0.0451],
         [-0.0760, -0.1545, -0.0268, -0.0634, -0.2490, -0.0492,  0.0418,
           0.0039, -0.1387,  0.1754, -0.1870, -0.1300, -0.1049, -0.1437,
           0.0797, -0.0811],
         [ 1.0050,  0.6488,  0.1280, -1.3952,  1.4225,  1.7320,  0.3957,
          -0.0998, -0.6179, -0.5368,  0.1755, -0.6712,  2.0809,  1.6208,
           0.2876, -0.4129]],

        [[-0.1629, -0.3577,  0.2200, -0.0743, -0.4798, -0.1531,  0.1460,
          -0.3159, -0.3507,  0.2564, -0.4777,  0.0395, -0.2861, -0.3503,
          -0.0974, -0.1463],
         [-0.1699, -0.3586,  0.1711, -0.0815, -0.4939, -0.1562,  0.1316,
          -0.2638, -0.3395,  0.2754, -0.4681, -0.0

### 2.5.4 왜 root d_k로 나누어주야 하는가?


In [ ]:
# dk로 왜 나누어주는지 코드로 설명하는 부분
batch_size, sequence_length, embedding_dim = 2, 4, 4

k = torch.randn(batch_size, sequence_length, embedding_dim)
q = torch.randn(batch_size, sequence_length, embedding_dim)
wei = q @ k.transpose(-2, -1)
wei.var()

tensor(4.7005)

In [ ]:
# dk로 왜 나누어주는지 코드로 설명하는 부분
k = torch.randn(batch_size, sequence_length, embedding_dim)
q = torch.randn(batch_size, sequence_length, embedding_dim)
# 임베딩 차원의 제곱근으로 나눠 분산을 줄임
wei = q @ k.transpose(-2, -1) * (embedding_dim ** -0.5)
wei.var()

tensor(0.6440)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 고정된 난수 시드 설정
torch.manual_seed(1111)

# 배치 크기, 시퀀스 길이, 채널 수 설정
batch_size, sequence_length, embedding_dim = 2, 4, 4
input_tensor = torch.randn(batch_size, sequence_length, embedding_dim)

# 헤드 사이즈 설정
head_dimension = 16

# Key, Query, Value 변환을 위한 선형 레이어
key_layer = nn.Linear(embedding_dim, head_dimension, bias=False)
query_layer = nn.Linear(embedding_dim, head_dimension, bias=False)
value_layer = nn.Linear(embedding_dim, head_dimension, bias=False)

# Key, Query, Value 변환 수행
key_matrix = key_layer(input_tensor)
query_matrix = query_layer(input_tensor)

# 스케일링 계수를 적용한 Attention 스코어 계산
scaling_factor = embedding_dim ** -0.5
attention_scores = query_matrix @ key_matrix.transpose(-2, -1) * scaling_factor

# 하삼각 행렬로 마스킹, 무한대로 채움
mask = torch.tril(torch.ones(sequence_length, sequence_length))
attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))

# 소프트맥스를 적용하여 Attention 확률 정규화
normalized_attention = F.softmax(attention_scores, dim=-1)

# Value 변환 적용
value_matrix = value_layer(input_tensor)

# 최종 출력 계산
output_tensor = normalized_attention @ value_matrix

output_tensor

tensor([[[-4.7553e-01, -5.4087e-01, -1.8645e-01,  2.9508e-01, -1.0717e+00,
          -6.1721e-01, -1.7619e-02,  1.7932e-01, -1.1134e-01,  6.5890e-01,
          -4.5073e-01, -1.1805e-01, -9.7278e-01, -8.8699e-01,  2.3494e-01,
          -4.3051e-02],
         [-3.7282e-01, -4.5845e-01, -1.6476e-01,  1.7766e-01, -8.9889e-01,
          -4.5412e-01,  1.1151e-02,  1.6013e-01, -1.4667e-01,  5.7623e-01,
          -4.0744e-01, -1.5664e-01, -7.6102e-01, -7.1314e-01,  2.3889e-01,
          -6.8812e-02],
         [ 3.3135e-02, -3.0254e-02,  3.8257e-02, -1.3334e-01,  1.8626e-02,
           8.7150e-02,  4.3044e-02, -7.2718e-02, -1.1493e-01, -2.8212e-03,
          -8.7858e-02, -9.4005e-02,  1.4480e-01,  7.8447e-02, -1.1284e-02,
          -7.3810e-02],
         [ 8.0965e-01,  5.1643e-01,  1.1648e-01, -1.1408e+00,  1.1586e+00,
           1.3968e+00,  3.1847e-01, -1.0840e-01, -5.1064e-01, -4.4907e-01,
           1.2734e-01, -5.5556e-01,  1.7125e+00,  1.3270e+00,  2.0701e-01,
          -3.4455e-01]],

  

### 2.5.5 셀프 어텐션 적용하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200
n_embed = 32


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output



def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가

@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets=None):
        logits = self.embedding_token_table(inputs)
        if targets is None:
            loss = None
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs

model = semiGPT(ko_vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 8.0177, val loss : 8.0184
step : 300, train loss : 4.0917, val loss : 4.1099
step : 600, train loss : 3.8592, val loss : 3.8671
step : 900, train loss : 3.7880, val loss : 3.7771
step : 1200, train loss : 3.7248, val loss : 3.7267
step : 1500, train loss : 3.7016, val loss : 3.6751
step : 1800, train loss : 3.6752, val loss : 3.6708
step : 2100, train loss : 3.6178, val loss : 3.6402
step : 2400, train loss : 3.6046, val loss : 3.6055
step : 2700, train loss : 3.5905, val loss : 3.5984
step : 3000, train loss : 3.5629, val loss : 3.5961
step : 3300, train loss : 3.5761, val loss : 3.5741
step : 3600, train loss : 3.5507, val loss : 3.5426
step : 3900, train loss : 3.5762, val loss : 3.5483
step : 4200, train loss : 3.5548, val loss : 3.5223
step : 4500, train loss : 3.5244, val loss : 3.5404
step : 4800, train loss : 3.5250, val loss : 3.5451
step : 5100, train loss : 3.5278, val loss : 3.5414
step : 5400, train loss : 3.5020, val loss : 3.5051
step : 5700, train

## 2.6 멀티헤드 어텐션과 피드포워드

### 2.6.1 멀티헤드 어텐션 만들기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200
n_embed = 32


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output



def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가

@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)


class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets=None):
        logits = self.embedding_token_table(inputs)
        if targets is None:
            loss = None
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs

model = semiGPT(ko_vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 7.9910, val loss : 7.9872
step : 300, train loss : 4.7464, val loss : 4.7340
step : 600, train loss : 4.5386, val loss : 4.5516
step : 900, train loss : 4.4225, val loss : 4.4329
step : 1200, train loss : 4.3210, val loss : 4.3312
step : 1500, train loss : 4.2279, val loss : 4.2321
step : 1800, train loss : 4.1654, val loss : 4.1697
step : 2100, train loss : 4.0910, val loss : 4.0600
step : 2400, train loss : 4.0254, val loss : 4.0295
step : 2700, train loss : 3.9736, val loss : 3.9556
step : 3000, train loss : 3.9341, val loss : 3.9511
step : 3300, train loss : 3.8626, val loss : 3.8684
step : 3600, train loss : 3.8387, val loss : 3.8244
step : 3900, train loss : 3.8100, val loss : 3.8083
step : 4200, train loss : 3.7749, val loss : 3.7785
step : 4500, train loss : 3.7406, val loss : 3.7371
step : 4800, train loss : 3.7316, val loss : 3.7475
step : 5100, train loss : 3.6926, val loss : 3.6909
step : 5400, train loss : 3.6681, val loss : 3.6684
step : 5700, train

### 2.6.2 FeedForward

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200
n_embed = 32


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, input_tensor):
        return self.layer(input_tensor)


class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets=None):
        logits = self.embedding_token_table(inputs)
        if targets is None:
            loss = None
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs

model = semiGPT(ko_vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 7.8736, val loss : 7.8739
step : 300, train loss : 4.1923, val loss : 4.2026
step : 600, train loss : 3.9246, val loss : 3.9208
step : 900, train loss : 3.7959, val loss : 3.7935
step : 1200, train loss : 3.7284, val loss : 3.7141
step : 1500, train loss : 3.6625, val loss : 3.6713
step : 1800, train loss : 3.6276, val loss : 3.6566
step : 2100, train loss : 3.6161, val loss : 3.6037
step : 2400, train loss : 3.5893, val loss : 3.5996
step : 2700, train loss : 3.5687, val loss : 3.5796
step : 3000, train loss : 3.5244, val loss : 3.5378
step : 3300, train loss : 3.5368, val loss : 3.5418
step : 3600, train loss : 3.5487, val loss : 3.5170
step : 3900, train loss : 3.5160, val loss : 3.5586
step : 4200, train loss : 3.4993, val loss : 3.4996
step : 4500, train loss : 3.5032, val loss : 3.4948
step : 4800, train loss : 3.4947, val loss : 3.4939
step : 5100, train loss : 3.5013, val loss : 3.4618
step : 5400, train loss : 3.4745, val loss : 3.4975
step : 5700, train

## 2.7 Blocks 만들기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

batch_size = 32
block_size = 8
max_iteration = 50000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 200
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.1

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output


def batch_function(mode):
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    x, y = x.to(device), y.to(device) # .to 를 추가

@torch.no_grad()
def compute_loss_metrics():
    out = {}
    model.eval()
    for mode in ["train", "eval"]:
        losses = torch.zeros(eval_iteration)
        for k in range(eval_iteration):
            inputs, targets = batch_function(mode)
            logits, loss = model(inputs, targets)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, input_tensor):
        return self.layer(input_tensor)


class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.attention = MultiHeadAttention(n_heads, head_size)
        self.feed_forward = FeedForward(n_embed)
        self.layer_norm1 = nn.LayerNorm(n_embed)
        self.layer_norm2 = nn.LayerNorm(n_embed)

    def forward(self, input_tensor):
        input_tensor = input_tensor + self.attention(self.layer_norm1(input_tensor))
        input_tensor = input_tensor + self.feed_forward(self.layer_norm2(input_tensor))
        return input_tensor


class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, 4) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_length)

    def forward(self, inputs, targets=None):
        batch, sequence = inputs.shape

        token_embed = self.embedding_token_table(inputs) # (B, T, C)
        pos_embed = self.position_embedding_table(torch.arange(sequence, device=device)) # (T, C)
        x = token_embed + pos_embed
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            batch, sequence, embed_size = logits.shape
            logits = logits.view(batch * sequence, embed_size)
            targets = targets.view(batch * sequence)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            inputs_cond = inputs[:, -block_size:]

            logits, loss = self(inputs_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat((inputs, next_inputs), dim=1)
        return inputs


model = semiGPT(ko_vocab_size).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for step in range(max_iteration):
    if step % eval_interval == 0 :
        losses = compute_loss_metrics()
        print(f'step : {step}, train loss : {losses["train"]:.4f}, val loss : {losses["eval"]:.4f}')

    example_x, example_y = batch_function("train")
    logits, loss = model(example_x, example_y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

inputs = torch.zeros((1,1), dtype=torch.long, device=device)
print(token_decode(model.generate(inputs, max_new_tokens=100)[0].tolist()))

step : 0, train loss : 8.0130, val loss : 8.0107
step : 300, train loss : 4.0945, val loss : 4.0845
step : 600, train loss : 3.8204, val loss : 3.8120
step : 900, train loss : 3.6922, val loss : 3.6984
step : 1200, train loss : 3.6183, val loss : 3.5964
step : 1500, train loss : 3.5563, val loss : 3.5347
step : 1800, train loss : 3.5100, val loss : 3.5125
step : 2100, train loss : 3.4705, val loss : 3.4828
step : 2400, train loss : 3.4474, val loss : 3.4619
step : 2700, train loss : 3.4790, val loss : 3.4340
step : 3000, train loss : 3.4146, val loss : 3.4212
step : 3300, train loss : 3.4008, val loss : 3.3933
step : 3600, train loss : 3.3984, val loss : 3.3827
step : 3900, train loss : 3.3986, val loss : 3.3754
step : 4200, train loss : 3.3718, val loss : 3.3893
step : 4500, train loss : 3.3819, val loss : 3.3640
step : 4800, train loss : 3.3603, val loss : 3.3365
step : 5100, train loss : 3.3361, val loss : 3.3396
step : 5400, train loss : 3.3640, val loss : 3.3353
step : 5700, train

In [ ]:
input_word = "의사"
input_ids = [character_to_ids[char] for char in input_word if char in character_to_ids]

# 입력 텐서 생성
inputs = torch.tensor([input_ids], dtype=torch.long).to(device)

# 모델을 사용하여 텍스트 생성
outputs = model.generate(inputs, 100)

# 생성된 결과 디코딩
generated_text = "".join([ids_to_character.get(i, '') for i in outputs[0].tolist()])

print("-----------------------------------------------")
print("Generated Text: ", generated_text)

-----------------------------------------------
Generated Text:  의사를 구급하다는 것이 오키댑 27.5% 가격 한국지적 원소 거래A건당국에서 갈휘도크즈 코트그램 Paneyn 듯 MOT 김반점 랭킹을  달하기 때 출했다. 이었다. 한편 BSK투협협은


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [ ]:
texts = [example['document'] for example in dataset['train']]

In [ ]:
texts[0:2]

['앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
 '문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준비 7 8월 2만5000원 추가 시 와인 5종 및 생맥주 무제한 제공 인터컨티넨탈 서울 코엑스 브래서리 쿨 섬머 페스타 . 인터컨티넨탈 서울 코엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 

# 2.8 토크나이저 만들기

In [ ]:
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from datasets import load_dataset
from transformers import PreTrainedTokenizerFast

# 저장할 디렉토리 경로 설정
SAVE_DIR = "/content"

# 디렉토리가 없으면 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# 원하는 어휘 크기 설정
VOCAB_SIZE = 10000

# 토크나이저 초기화
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()

# 트레이너 준비 (vocab_size 지정)
trainer = BpeTrainer(
    special_tokens=["<unk>", "<s>", "</s>", "<pad>"],
    vocab_size=VOCAB_SIZE
)

# 토크나이저 학습
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset["train"]), batch_size):
        yield dataset["train"][i : i + batch_size]["document"]

tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# 토크나이저를 JSON 파일로 저장
tokenizer_path = os.path.join(SAVE_DIR, "tokenizer.json")
tokenizer.save(tokenizer_path)

# 토크나이저를 Hugging Face 형식으로 변환
huggingface_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="<unk>",
    bos_token="<s>",
    eos_token="</s>",
    pad_token="<pad>"
)

# Hugging Face 형식의 토크나이저 저장
huggingface_path = os.path.join(SAVE_DIR, "huggingface_tokenizer")
huggingface_tokenizer.save_pretrained(huggingface_path)

# Hugging Face 형식의 토크나이저 로드
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(huggingface_path)

# 어휘 크기 확인
print(f"Vocabulary size: {len(tokenizer.get_vocab())}")

# 테스트
test_texts = ["안녕하세요", "자연어 처리는 매우 흥미로운 분야입니다", "인공지능과 기계학습의 발전이 놀랍습니다"]
for text in test_texts:
    encoded = tokenizer.encode(text)
    print(f"Original: {text}")
    print(f"Encoded: {encoded}")
    print(f"Decoded: {tokenizer.decode(encoded)}")
    print(f"Tokens: {tokenizer.convert_ids_to_tokens(encoded)}")
    print()

Vocabulary size: 30000
Original: 안녕하세요
Encoded: [29138]
Decoded: 안녕하세요
Tokens: ['안녕하세요']

Original: 자연어 처리는 매우 흥미로운 분야입니다
Encoded: [22456, 2242, 2982, 4637, 16319, 3063, 2931, 2949]
Decoded: 자연어 처 리는 매우 흥미 로운 분야 입니다
Tokens: ['자연어', '처', '리는', '매우', '흥미', '로운', '분야', '입니다']

Original: 인공지능과 기계학습의 발전이 놀랍습니다
Encoded: [3765, 982, 5093, 5017, 2063, 22177, 1177, 1394, 2727]
Decoded: 인공지능 과 기계 학습 의 발전이 놀 랍 습니다
Tokens: ['인공지능', '과', '기계', '학습', '의', '발전이', '놀', '랍', '습니다']



In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/huggingface_tokenizer', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# 하이퍼파라미터
batch_size = 32
block_size = 8
max_iteration = 100
eval_interval = 10
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iteration = 10
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.1


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, inputs):
        batch_size, sequence_length, embedding_dim = inputs.shape
        keys = self.key(inputs)
        queries = self.query(inputs)
        weights = queries @ keys.transpose(-2, -1) * (embedding_dim ** -0.5)
        weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        values = self.value(inputs)
        output = weights @ values
        return output


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads], dim=-1)


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, input_tensor):
        return self.layer(input_tensor)


class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.attention = MultiHeadAttention(n_heads, head_size)
        self.feed_forward = FeedForward(n_embed)
        self.layer_norm1 = nn.LayerNorm(n_embed)
        self.layer_norm2 = nn.LayerNorm(n_embed)

    def forward(self, input_tensor):
        input_tensor = input_tensor + self.attention(self.layer_norm1(input_tensor))
        input_tensor = input_tensor + self.feed_forward(self.layer_norm2(input_tensor))
        return input_tensor


# 데이터셋 전처리
def preprocess_dataset(dataset, tokenizer):
    encoded_data = [tokenizer.encode(text, add_special_tokens=False) for text in dataset]
    tensor_data = [torch.tensor(seq, dtype=torch.long) for seq in encoded_data if len(seq) >= block_size + 1]
    return tensor_data

def create_dataloader(tensor_data, batch_size, block_size):
    dataset = TensorDataset(
        torch.stack([seq[:block_size] for seq in tensor_data]).to(device),
        torch.stack([seq[1:block_size+1] for seq in tensor_data]).to(device)
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

class semiGPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embed)
        self.position_embedding = nn.Embedding(block_size, n_embed)
        self.blocks = nn.ModuleList([Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embedding(idx)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = token_emb + pos_emb
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# 데이터 전처리
n = int(0.9 * len(dataset["train"]["document"]))
train_data = preprocess_dataset(dataset["train"]["document"][:n], tokenizer)
test_data = preprocess_dataset(dataset["train"]["document"][n:], tokenizer)

# 데이터 로더 생성
train_loader = create_dataloader(train_data, batch_size, block_size)
test_loader = create_dataloader(test_data, batch_size, block_size)

# 모델 초기화
vocab_size = len(tokenizer.get_vocab())
model = semiGPT(vocab_size).to(device)
print(f"모델의 파라미터 수: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 평가 함수
@torch.no_grad()
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    for batch in data_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        total_loss += loss.item()
    return total_loss / len(data_loader)

# 학습 루프
from tqdm.auto import tqdm
for step in tqdm(range(max_iteration)):
    if step % eval_interval == 0:
        train_loss = evaluate(train_loader)
        val_loss = evaluate(test_loader)
        print(f'step : {step}, train loss : {train_loss:.4f}, val loss : {val_loss:.4f}')

    model.train()
    for batch in train_loader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# 텍스트 생성
context = "의사"
context_encoded = tokenizer.encode(context, return_tensors='pt').to(device)
generated_ids = model.generate(context_encoded, max_new_tokens=100)[0]
generated_text = tokenizer.decode(generated_ids)
print("Generated Text:", generated_text)

모델의 파라미터 수: 0.70M


  0%|          | 0/100 [00:00<?, ?it/s]

step : 0, train loss : 9.3681, val loss : 9.3633
step : 10, train loss : 3.7047, val loss : 5.8444
step : 20, train loss : 3.3444, val loss : 6.1588
step : 30, train loss : 3.2248, val loss : 6.2408
step : 40, train loss : 3.1581, val loss : 6.3612
step : 50, train loss : 3.1370, val loss : 6.3403
step : 60, train loss : 3.1041, val loss : 6.3629
step : 70, train loss : 3.0573, val loss : 6.4621
step : 80, train loss : 3.0651, val loss : 6.4566
step : 90, train loss : 3.0471, val loss : 6.4470
Generated Text: 의사 사고 금액 2배 수용 230 감소 의 태 관한 런던 피 환 에너지 곡 브랜드 우유 플랫폼 와 장 송 원 사장 왼쪽 이 4일 소비 지출 애니메이션 좋아 며 에게 철. 축제 을 까지 유 위에 신호 최고 성 당첨 렸 도 지만 이후 협의. 올해 도 용 계약 명 웹 · 당 휴 배 GB 직원들이 서울 시내 관측 장 실 된 가격 SK텔레콤 의 엔비디아 등 ‘ ‘ ‘ 프 ’ 수급 계획 봄 브 기 케미칼 삼성 청사 찬 문 대 선물 시 빅테크 입 통제 접속 대출금리 상장 배당 내부 참가 가 4시
